# Workshop: Transformations & Data Cleaning

**Cel szkoleniowy:** Praktyczne opanowanie transformacji danych w PySpark i SQL oraz technik czyszczenia danych.

**Zakres tematyczny:**
- Transformacje kolumnowe i warunkowe
- Filtry, sortowania, agregacje
- Porównanie PySpark vs SQL
- Czyszczenie: nulls, duplikaty, walidacja typów
- Quality checks i flagowanie problemów

**Czas trwania:** 90 minut

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 1 - Fundamentals & Exploration
- **Typ notebooka**: Workshop
- **Wymagania techniczne**:
 - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
 - Unity Catalog włączony
 - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
 - Klaster: Standard z minimum 2 workers

## Wprowadzenie do warsztatu

W tym warsztacie będziesz:
1. Transformować dane za pomocą PySpark DataFrame API
2. Pisać równoważne transformacje w SQL
3. Czyścić dane (nulls, duplikaty, nieprawidłowe wartości)
4. Walidować typy danych i konwertować kolumny
5. Implementować quality checks

### Kryteria sukcesu:
- Wszystkie transformacje działają poprawnie w PySpark i SQL
- Dane są oczyszczone z duplikatów i null values
- Quality checks zidentyfikowały wszystkie problemy
- Finalna tabela spełnia standardy jakości

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [ ]:
%run ../../00_setup

## Konfiguracja

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, trim, lower, upper, regexp_replace, to_date, coalesce, lit
from pyspark.sql.types import *

# Ścieżki do danych
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

**Konfiguracja środowiska wykonana**

Biblioteki i ścieżki zostały skonfigurowane:
- **Funkcje PySpark**: Transformacje, filtry, agregacje 
- **Katalog**: `{CATALOG}`
- **Schema**: `{BRONZE_SCHEMA}` (izolacja per użytkownik)
- **Źródła danych**: customers.csv, orders_batch.json

## Przygotowanie danych

In [ ]:
# Wczytaj dane customers
customers_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(CUSTOMERS_CSV)
customers_df.count()

In [ ]:
# Wczytaj dane orders
orders_df = spark.read.format("json").option("inferSchema", "true").load(ORDERS_JSON)
orders_df.count()

**Dane wczytane pomyślnie**

Oba datasety zostały wczytane:
- **Customers**: ~10,000 rekordów z pliku CSV
- **Orders**: ~100,000 rekordów z pliku JSON

**Rzeczywiste kolumny:**
- **Customers**: `customer_id`, `first_name`, `last_name`, `email`, `phone`, `city`, `state`, `country`, `registration_date`, `customer_segment`
- **Orders**: `order_id`, `customer_id`, `product_id`, `store_id`, `order_datetime`, `quantity`, `unit_price`, `discount_percent`, `total_amount`, `payment_method`

---

## Zadanie 1: Transformacje kolumnowe - PySpark (15 min)

### Cel:
Zastosuj transformacje kolumnowe na danych klientów.

### Instrukcje:
1. Dodaj kolumnę `email_domain` wyodrębniając domenę z email
2. Dodaj kolumnę `country_code` z pierwszymi 2 literami kraju (uppercase)
3. Dodaj kolumnę `customer_full_name` - trim i lowercase
4. Usuń kolumnę `registration_date` (jeśli istnieje)
5. Zmień nazwę kolumny `first_name` na `customer_full_name`

### Wskazówki:
- Użyj: `withColumn()`, `regexp_extract()`, `substring()`, `trim()`, `lower()`, `drop()`, `withColumnRenamed()`

In [ ]:
# Dodaj kolumnę email_domain
customers_transformed = customers_df.withColumn(
 "email_domain",
 F.____(____(____), "@(.+)$", 1) # regexp_extract, col, "email"
)

display(customers_transformed.select("customer_id", "email", "email_domain").limit(5))

# TODO: Dodaj kolumnę country_code (pierwsze 2 litery, uppercase)
customers_transformed = customers_transformed.withColumn(
 "country_code",
 ____(col("____").substr(1, ____)) # upper, country, 2
)

# TODO: Dodaj kolumnę customer_full_name (trim i lowercase)
customers_transformed = customers_transformed.withColumn(
 "customer_full_name",
 ____(____(col("first_name"))) # lower, trim
)

# TODO: Usuń kolumnę registration_date (jeśli istnieje)
if "registration_date" in customers_transformed.columns:
 customers_transformed = customers_transformed.____("____") # drop, registration_date

# TODO: Zmień nazwę first_name na customer_full_name
customers_transformed = customers_transformed.____("first_name", "____") # withColumnRenamed, customer_full_name

print("=== Przykładowe dane po transformacjach ===")
display(customers_transformed.limit(5))

**Transformacja email_domain wykonana**

Za pomocą `regexp_extract()` wyodrębniamy domenę email po znaku `@`.
- **Regex pattern**: `@(.+)$` - wszystko po `@` do końca stringa
- **Group 1**: Pierwsza grupa w nawiasach `(.+)`

**Rezultat**: Nowa kolumna `email_domain` z domenami email (gmail.com, interia.pl, etc.)

In [ ]:
# Dodaj kolumnę country_code (pierwsze 2 litery, uppercase)
customers_transformed = customers_transformed.withColumn(
 "country_code",
 ____(col("____").substr(1, ____)) # upper, "country", 2
)

display(customers_transformed.select("customer_id", "country", "country_code").limit(5))

**Transformacja country_code wykonana**

Tworzenie kodu kraju z pierwszych 2 liter:
- **substr(1, 2)**: Pierwsze 2 znaki z nazwy kraju
- **upper()**: Konwersja na wielkie litery

**Rezultat**: "USA" -> "US", "Poland" -> "PO", "Germany" -> "GE"

In [ ]:
# Dodaj kolumnę full_name (łącząc imię i nazwisko)
customers_transformed = customers_transformed.withColumn(
 "full_name",
 ____(____(col("____") + " " + col("____"))) # trim, lower, "first_name", "last_name"
)

display(customers_transformed.select("customer_id", "first_name", "last_name", "full_name").limit(5))

**Transformacja full_name wykonana**

Łączenie imienia i nazwiska w jedną kolumnę:
- **Konkatenacja**: `first_name + " " + last_name`
- **trim()**: Usuwa zbędne spacje
- **lower()**: Konwersja na małe litery dla spójności

**Rezultat**: "Jesse Hoffman" -> "jesse hoffman"

---

## Zadanie 2: Transformacje warunkowe (15 min)

### Cel:
Dodaj kolumny z logiką warunkową.

### Instrukcje:
1. Dodaj kolumnę `customer_segment` (VIP dla PL/DE, Regular dla reszty)
2. Dodaj kolumnę `email_valid` (True jeśli email zawiera '@', False w przeciwnym razie)
3. Dodaj kolumnę `country_region` (Europe dla PL/DE/FR, Other dla reszty)

### Wskazówki:
- Użyj: `when().otherwise()`
- Dla wielu warunków: `.when().when().otherwise()`

In [ ]:
# Dodaj kolumnę customer_tier (VIP dla USA, Regular dla reszty)
customers_transformed = customers_transformed.withColumn(
 "customer_tier",
 when(col("country") == "USA", "VIP") # when, "USA"
 .otherwise("Regular") # "Regular"
)

display(customers_transformed.select("customer_id", "country", "customer_tier").limit(10))

# TODO: Dodaj kolumnę customer_segment (VIP dla PL/DE, Regular dla reszty)
customers_transformed = customers_transformed.withColumn(
 "customer_segment",
 when(col("country").isin("Poland", "Germany"), "VIP") # when, Germany
 .otherwise("Regular") # Regular
)

# TODO: Dodaj kolumnę email_valid (True jeśli zawiera @)
customers_transformed = customers_transformed.withColumn(
 "email_valid",
 when(col("email").contains("@"), True) # email, contains, True
 .otherwise(False)
)

# TODO: Dodaj kolumnę country_region (Europe dla PL/DE/FR, Other dla reszty)
customers_transformed = customers_transformed.withColumn(
 "country_region",
 when(col("country").isin("Poland", "Germany", "France"), "Europe") # isin, Europe
 .otherwise("Other") # otherwise
)

print("=== Przykładowe dane z transformacjami warunkowymi ===")
display(
 customers_transformed
 .select("customer_id", "country", "customer_segment", "email_valid", "country_region")
 .limit(10)
)

**Segmentacja klientów customer_tier**

Logika biznesowa segmentacji:
- **VIP**: Klienci z USA (główny rynek)
- **Regular**: Wszyscy pozostali klienci

**when().otherwise()**: Struktura warunkowa PySpark podobna do CASE WHEN w SQL

In [ ]:
# Dodaj kolumnę email_valid (walidacja obecności @)
customers_transformed = customers_transformed.withColumn(
 "email_valid",
 when(col("____").____("@"), ____) # "email", contains, True
 .otherwise(____) # False
)

display(customers_transformed.select("customer_id", "email", "email_valid").limit(10))

**Walidacja email_valid**

Prosta walidacja formatu email:
- **True**: Email zawiera znak `@`
- **False**: Brak znaku `@` (nieprawidłowy format)

**Zastosowanie biznesowe**: Flagowanie niepoprawnych adresów email do ręcznej weryfikacji lub czyszczenia

---

## Zadanie 3: Filtry i sortowanie (10 min)

### Cel:
Filtruj i sortuj dane.

### Instrukcje:
1. Filtruj klientów z Polski
2. Filtruj zamówienia o wartości > 100
3. Sortuj zamówienia według total_amount malejąco
4. Znajdź top 10 najdroższych zamówień

### Wskazówki:
- Użyj: `.filter()` lub `.where()`, `.orderBy()`, `.limit()`

In [ ]:
# Filtruj klientów z USA
usa_customers = customers_transformed.filter(col("____") == "USA") # "country"
usa_customers.count()

# TODO: Filtruj klientów z Polski
polish_customers = customers_transformed.filter(col("____") == "Poland") # country
print(f"Klienci z Polski: {polish_customers.count()}")

# TODO: Filtruj zamówienia o wartości > 100
high_value_orders = orders_df.filter(col("____") > ____) # total_amount, 100
print(f"Zamówienia > 100: {high_value_orders.count()}")

# TODO: Top 10 najdroższych zamówień (sortowanie malejące)
top_orders = (
 orders_df
 .orderBy(col("total_amount").____()) # desc
 .limit(____) # 10
)

print("\n=== Top 10 najdroższych zamówień ===")
display(top_orders)

In [ ]:
# Filtruj zamówienia o wartości > 500
high_value_orders = orders_df.filter(col("____") > ____) # "total_amount", 500
high_value_orders.count()

In [ ]:
# Top 10 najdroższych zamówień (sortowanie malejące)
top_orders = (
 orders_df
 .orderBy(col("total_amount").____()) # desc
 .limit(____) # 10
)

display(top_orders.select("order_id", "customer_id", "total_amount", "payment_method"))

**Operacje filtrowania i sortowania wykonane**

Zastosowane filtry:
- **USA customers**: Filtrowanie po kraju równym "USA"
- **High-value orders**: Zamówienia > 500 (próg premium)
- **Top 10 orders**: Sortowanie malejące po `total_amount`

**Metody**: `filter()`, `orderBy()`, `limit()` - podstawowe operacje analityczne w PySpark

---

## Zadanie 4: Agregacje - PySpark (15 min)

### Cel:
Przeprowadź agregacje na zamówieniach.

### Instrukcje:
1. Policz zamówienia według statusu
2. Oblicz całkowitą wartość zamówień według statusu
3. Znajdź średnią wartość zamówienia według customer_id
4. Policz zamówienia per klient

### Wskazówki:
- Użyj: `.groupBy()`, `.agg()`, `count()`, `sum()`, `avg()`

In [ ]:
# Policz zamówienia według metod płatności
orders_by_payment = (
 orders_df
 .groupBy("____") # "payment_method"
 .count()
 .orderBy("count", ascending=____) # False
)

display(orders_by_payment)

**Agregacja według metod płatności**

Analiza preferencji płatniczych klientów:
- **groupBy("payment_method")**: Grupowanie według sposobu płatności
- **count()**: Liczba zamówień w każdej grupie
- **orderBy(desc)**: Sortowanie od najczęstszej metody

**Biznesowa wartość**: Identyfikacja popularnych metod płatności dla optymalizacji procesu checkout

In [ ]:
# Całkowita wartość i średnia według metod płatności
revenue_by_payment = (
 orders_df
 .groupBy("payment_method")
 .agg(
 F.____("total_amount").alias("total_revenue"), # sum
 F.____("total_amount").alias("avg_order_value") # avg
 )
 .orderBy("total_revenue", ascending=False)
)

display(revenue_by_payment)

**Analiza przychodów według metod płatności**

Zaawansowana agregacja z wieloma miarami:
- **F.sum()**: Całkowity przychód per metoda płatności
- **F.avg()**: Średnia wartość zamówienia (AOV)
- **alias()**: Nadanie czytelnych nazw kolumnom wynikowym

**Insight**: Która metoda płatności generuje największy przychód?

In [ ]:
# Statystyki per klient
customer_stats = (
 orders_df
 .groupBy("____") # "customer_id"
 .agg(
 F.____("*").alias("total_orders"), # count
 F.sum("____").alias("total_spent"), # "total_amount"
 F.avg("total_amount").alias("avg_order_value")
 )
 .orderBy("____", ascending=False) # "total_spent"
 .limit(10)
)

display(customer_stats)

**Top 10 klientów - analiza RFM**

Kluczowe metryki klientów:
- **total_orders**: Frequency - jak często kupuje
- **total_spent**: Monetary - ile wydaje łącznie 
- **avg_order_value**: Średnia wartość koszyka

**Zastosowanie**: Identyfikacja VIP customers, programy lojalnościowe, personalizacja ofert

---

## Zadanie 5: SQL Equivalent (10 min)

### Cel:
Napisz te same transformacje w SQL.

### Instrukcje:
1. Utwórz temp view dla customers i orders
2. Napisz SQL query agregujący zamówienia według statusu
3. Napisz SQL query z JOINem customers i orders

### Wskazówki:
- Użyj: `createOrReplaceTempView()`, `spark.sql()`

In [ ]:
# Utwórz temp views dla SQL queries
customers_df.____("customers") # createOrReplaceTempView
orders_df.createOrReplaceTempView("____") # "orders"

**Temporary Views utworzone**

Views pozwalają na używanie SQL na DataFrames:
- **customers**: DataFrame customers_df jako tabela SQL
- **orders**: DataFrame orders_df jako tabela SQL

**Zasięg**: Sesja Spark (znikają po restarcie klastra)

In [ ]:
# SQL query - agregacja zamówień według metod płatności
result = spark.sql("""
 SELECT 
 payment_method,
 ____(*) as order_count, -- COUNT
 ____(total_amount) as total_revenue, -- SUM
 ____(total_amount) as avg_order_value -- AVG
 FROM ____ -- orders
 GROUP BY ____ -- payment_method
 ORDER BY total_revenue ____ -- DESC
""")

print("=== Agregacja zamówień (SQL) ===")
display(result)

**SQL Aggregation - identyczna logika jak PySpark**

SQL wersja poprzedniej agregacji PySpark:
- **SELECT**: Wybór kolumn i funkcji agregujących
- **GROUP BY**: Grupowanie po payment_method
- **ORDER BY**: Sortowanie wyników

**Rezultat**: Identyczne wyniki jak `groupBy().agg()` w PySpark

In [ ]:
# SQL query - JOIN customers i orders
result = spark.sql("""
 SELECT 
 c.customer_id,
 c.first_name,
 c.last_name,
 c.country,
 ____(____.order_id) as total_orders, -- COUNT, o
 SUM(o.____) as total_spent -- total_amount
 FROM ____ c -- customers
 ____ JOIN orders o ON c.customer_id = o.customer_id -- LEFT
 GROUP BY c.customer_id, c.first_name, c.last_name, c.country
 ORDER BY ____ DESC -- total_spent
 LIMIT ____ -- 10
""")

display(result)

**SQL JOIN - łączenie tabel customers i orders**

Zaawansowane SQL z JOINem:
- **LEFT JOIN**: Wszyscy klienci + ich zamówienia (jeśli istnieją)
- **GROUP BY**: Agregacja per klient
- **ORDER BY + LIMIT**: Top 10 klientów według wydatków

**Equivalent PySpark**: `customers_df.join(orders_df, "customer_id", "left").groupBy(...)`

---

## Zadanie 6: Data Cleaning - Nulls (15 min)

### Cel:
Obsłuż wartości NULL w danych.

### Instrukcje:
1. Policz wartości NULL w każdej kolumnie customers
2. Zastąp NULL w `email` wartością "unknown@example.com"
3. Zastąp NULL w `country` wartością "Unknown"
4. Usuń rekordy gdzie `customer_id` jest NULL

### Wskazówki:
- Użyj: `.fillna()`, `.dropna()`, `coalesce()`

In [ ]:
# TODO: Policz wartości NULL w każdej kolumnie
from pyspark.sql.functions import sum as spark_sum

null_counts = customers_df.select([
 spark_sum(col(c).____().cast("int")).alias(c) # isNull
 for c in customers_df.columns
])

print("=== Wartości NULL przed czyszczeniem ===")
display(null_counts)

In [ ]:
# TODO: Zastąp NULL w email wartością domyślną
customers_clean = customers_df.____({ # fillna
 "email": "____" # unknown@example.com
})

# TODO: Zastąp NULL w country
customers_clean = customers_clean.fillna({
 "____": "Unknown" # country
})

# TODO: Usuń rekordy gdzie customer_id jest NULL
customers_clean = customers_clean.____(____ =["customer_id"]) # dropna, subset

print(f"\n Rekordy po czyszczeniu NULL: {customers_clean.count()}")
print(f" Usunięto: {customers_df.count() - customers_clean.count()} rekordów")

---

## Zadanie 7: Deduplikacja (10 min)

### Cel:
Usuń duplikaty z danych.

### Instrukcje:
1. Znajdź liczbę duplikatów w customers (wszystkie kolumny)
2. Usuń duplikaty oparte na customer_id
3. Usuń duplikaty oparte na email

### Wskazówki:
- Użyj: `.dropDuplicates()`, `.distinct()`

In [ ]:
# TODO: Znajdź liczbę duplikatów (wszystkie kolumny)
total_rows = customers_clean.count()
distinct_rows = customers_clean.____().count() # distinct
duplicates = total_rows - distinct_rows

print("=== Analiza duplikatów ===")
print(f"Całkowita liczba rekordów: {total_rows}")
print(f"Unikalne rekordy: {distinct_rows}")
print(f"Duplikaty (wszystkie kolumny): {____}") # duplicates

In [ ]:
# TODO: Usuń duplikaty na podstawie customer_id
customers_dedup = customers_clean.____([____]) # dropDuplicates, "customer_id"

print(f"\n Rekordy po deduplikacji (customer_id): {customers_dedup.count()}")
print(f" Usunięto: {customers_clean.count() - customers_dedup.count()} duplikatów")

In [ ]:
# TODO: Sprawdź duplikaty w kolumnie email
email_duplicates = (
 customers_clean
 .groupBy("____") # email
 .count()
 .filter("count > ____") # 1
)

print(f"\n=== Duplikaty email ===")
print(f"Liczba duplikatów email: {email_duplicates.count()}")

if email_duplicates.count() > 0:
 print("\nPrzykłady duplikatów email:")
 display(email_duplicates.orderBy("count", ascending=False).limit(5))

---

## Zadanie 8: Quality Checks (10 min)

### Cel:
Zaimplementuj quality checks i flaguj problemy.

### Instrukcje:
1. Dodaj kolumnę `quality_flag` (0 = OK, 1 = problem)
2. Flaguj rekordy gdzie email nie zawiera '@'
3. Flaguj rekordy gdzie country = "Unknown"
4. Policz rekordy z problemami

### Wskazówki:
- Użyj: `when().otherwise()` dla flagowania

In [ ]:
# TODO: Dodaj quality_flag (0 = OK, 1 = problem)
customers_with_flags = customers_dedup.withColumn(
 "quality_flag",
 when(~col("email").____("@"), ____) # contains, 1
 .when(col("____") == "Unknown", 1) # country
 .otherwise(____) # 0
)

# TODO: Policz rekordy z problemami
problem_count = customers_with_flags.filter(col("quality_flag") == ____).count() # 1
ok_count = customers_with_flags.filter(col("quality_flag") == 0).count()

print("=== Quality Checks ===")
print(f" Rekordy OK: {ok_count}")
print(f" Rekordy z problemami: {____}") # problem_count
print(f" Procent problemów: {(problem_count / customers_with_flags.count() * 100):.2f}%")

In [ ]:
# TODO: Wyświetl przykładowe rekordy z problemami
if problem_count > 0:
 print("\n=== Przykładowe rekordy z problemami ===")
 display(
 customers_with_flags
 .filter(col("____") == 1) # quality_flag
 .select("customer_id", "customer_full_name", "email", "country", "quality_flag")
 .limit(5)
 )
else:
 print("\n Brak rekordów z problemami!")

---

## Walidacja i weryfikacja

### Checklist:
- [ ] Transformacje kolumnowe zastosowane
- [ ] Transformacje warunkowe działają
- [ ] Agregacje poprawne (PySpark i SQL)
- [ ] NULL values obsłużone
- [ ] Duplikaty usunięte
- [ ] Quality checks zaimplementowane

In [ ]:
print("="*60)
print("=== WERYFIKACJA WYNIKÓW WARSZTATU ===")
print("="*60)

try:
 # Sprawdź czy wszystkie DataFrames zostały utworzone
 assert 'customers_transformed' in locals(), "customers_transformed nie został utworzony"
 assert 'customers_clean' in locals(), "customers_clean nie został utworzony"
 assert 'customers_dedup' in locals(), "customers_dedup nie został utworzony"
 assert 'customers_with_flags' in locals(), "customers_with_flags nie został utworzony"
 
 print("\n Wszystkie DataFrames utworzone")
 
 # Statystyki finalne
 print(f"\n📊 Statystyki końcowe:")
 print(f" Oryginalna liczba rekordów: {customers_df.count()}")
 print(f" Po transformacjach: {customers_transformed.count()}")
 print(f" Po czyszczeniu NULL: {customers_clean.count()}")
 print(f" Po deduplikacji: {customers_dedup.count()}")
 print(f" Finalna liczba rekordów: {customers_with_flags.count()}")
 
 print(f"\n Quality Checks:")
 ok_records = customers_with_flags.filter(col('quality_flag') == 0).count()
 problem_records = customers_with_flags.filter(col('quality_flag') == 1).count()
 print(f" Rekordy bez problemów: {ok_records}")
 print(f" Rekordy z problemami: {problem_records}")
 
 # Zapisz do tabeli (opcjonalnie)
 output_table = f"{BRONZE_SCHEMA}.customers_clean_workshop"
 print(f"\n💾 Zapisywanie do tabeli: {output_table}")
 
 customers_with_flags.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(output_table)
 
 print(f" Zapisano tabelę: {output_table}")
 
 print("\n" + "="*60)
 print(" WARSZTAT ZAKOŃCZONY POMYŚLNIE! ")
 print("="*60)
 
except AssertionError as e:
 print(f"\n✗ Błąd: {e}")
 print(" Upewnij się, że wszystkie zadania zostały wykonane!")
except Exception as e:
 print(f"\n✗ Nieoczekiwany błąd: {e}")

---

## Podsumowanie warsztatu

### Co osiągnąłeś:

**Transformacje:**
- Transformacje kolumnowe: `withColumn()`, `drop()`, `withColumnRenamed()`
- Ekstrakcja danych: `regexp_extract()`, `substring()`
- Formatowanie: `trim()`, `lower()`, `upper()`
- Logika warunkowa: `when().otherwise()`
- Filtry i sortowanie: `filter()`, `orderBy()`, `limit()`

**Agregacje:**
- Grupowanie: `groupBy()`, `agg()`
- Funkcje agregujące: `count()`, `sum()`, `avg()`, `min()`, `max()`
- Równoważne zapytania SQL

**Data Cleaning:**
- Obsługa wartości NULL: `fillna()`, `dropna()`, `coalesce()`
- Deduplikacja: `dropDuplicates()`, `distinct()`
- Quality checks i flagowanie problemów
- Analiza jakości danych

### Kluczowe wnioski:

1. **PySpark vs SQL** - Oba podejścia dają te same rezultaty, wybór zależy od preferencji
2. **Czyszczenie danych jest kluczowe** - NULL i duplikaty mogą zniekształcić analizy
3. **Quality flags** - Pozwalają na monitorowanie jakości bez usuwania danych
4. **Transformacje łańcuchowe** - PySpark pozwala na eleganckie łączenie operacji

### 📚 Best Practices zastosowane:

- Dodawanie nowych kolumn zamiast modyfikowania istniejących
- Logowanie statystyk przed i po czyszczeniu
- Używanie flag jakości zamiast usuwania problematycznych danych
- Komentowanie i dokumentowanie transformacji
- Weryfikacja wyników na każdym etapie

### ➡ Następne kroki:
- **Kolejny warsztat**: `03_views_basic_jobs_workshop.ipynb`
- **Praktyka**: Zastosuj te techniki na własnych danych
- **Dokumentacja**: [PySpark SQL Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html)

---